In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean, cityblock, chebyshev
from scipy.stats import chi2_contingency
import pandas as pd

def extract_color_histogram(image, bins=32, color_space='RGB'):
    """
    이미지에서 컬러 히스토그램 추출
    
    Args:
        image: PIL Image 또는 numpy array
        bins: 히스토그램 빈의 수
        color_space: 색상 공간 ('RGB', 'HSV', 'LAB')
    
    Returns:
        normalized histogram (flatten)
    """
    # PIL Image를 numpy array로 변환
    if isinstance(image, Image.Image):
        image = np.array(image)
    
    # 색상 공간 변환
    if color_space == 'HSV':
        image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    elif color_space == 'LAB':
        image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    # RGB는 변환 불필요
    
    # 각 채널별 히스토그램 계산
    hist_r = cv2.calcHist([image], [0], None, [bins], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [bins], [0, 256])
    hist_b = cv2.calcHist([image], [2], None, [bins], [0, 256])
    
    # 히스토그램 결합 및 정규화
    combined_hist = np.concatenate([hist_r, hist_g, hist_b]).flatten()
    normalized_hist = combined_hist / (combined_hist.sum() + 1e-7)  # 0으로 나누기 방지
    
    return normalized_hist

def calculate_histogram_similarity(hist1, hist2, method='correlation'):
    """
    두 히스토그램 간의 유사도 계산
    
    Args:
        hist1, hist2: 정규화된 히스토그램
        method: 유사도 계산 방법
    
    Returns:
        similarity score
    """
    if method == 'correlation':
        # OpenCV 상관관계 (높을수록 유사)
        return cv2.compareHist(hist1.astype(np.float32), hist2.astype(np.float32), cv2.HISTCMP_CORREL)
    
    elif method == 'chi_square':
        # 카이제곱 거리 (낮을수록 유사)
        return cv2.compareHist(hist1.astype(np.float32), hist2.astype(np.float32), cv2.HISTCMP_CHISQR)
    
    elif method == 'intersection':
        # 히스토그램 교집합 (높을수록 유사)
        return cv2.compareHist(hist1.astype(np.float32), hist2.astype(np.float32), cv2.HISTCMP_INTERSECT)
    
    elif method == 'bhattacharyya':
        # 바타차리야 거리 (낮을수록 유사)
        return cv2.compareHist(hist1.astype(np.float32), hist2.astype(np.float32), cv2.HISTCMP_BHATTACHARYYA)
    
    elif method == 'cosine':
        # 코사인 유사도 (높을수록 유사)
        return cosine_similarity([hist1], [hist2])[0][0]
    
    elif method == 'euclidean':
        # 유클리드 거리 (낮을수록 유사)
        return euclidean(hist1, hist2)
    
    elif method == 'manhattan':
        # 맨하탄 거리 (낮을수록 유사)
        return cityblock(hist1, hist2)

def extract_style_histogram(folder_path, bins=32, color_space='RGB', max_images=None):
    """
    특정 스타일 폴더의 모든 이미지에서 평균 히스토그램 추출
    
    Args:
        folder_path: 이미지 폴더 경로
        bins: 히스토그램 빈의 수
        color_space: 색상 공간
        max_images: 처리할 최대 이미지 수 (None이면 모든 이미지)
    
    Returns:
        averaged and normalized histogram
    """
    all_histograms = []
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    
    if max_images:
        image_files = image_files[:max_images]
    
    for fname in tqdm(image_files, desc=f"Processing {os.path.basename(folder_path)}"):
        fpath = os.path.join(folder_path, fname)
        try:
            image = Image.open(fpath).convert('RGB')
            # 이미지 크기 조정 (처리 속도 향상)
            image = image.resize((256, 256))
            hist = extract_color_histogram(image, bins=bins, color_space=color_space)
            all_histograms.append(hist)
        except Exception as e:
            print(f"Error processing {fpath}: {e}")
    
    if not all_histograms:
        return None
    
    # 모든 히스토그램의 평균 계산
    mean_histogram = np.mean(all_histograms, axis=0)
    return mean_histogram

def create_similarity_matrix(style_histograms, method='correlation'):
    """
    스타일 간 유사도 매트릭스 생성
    
    Args:
        style_histograms: 스타일별 히스토그램 딕셔너리
        method: 유사도 계산 방법
    
    Returns:
        similarity matrix, style names
    """
    styles = list(style_histograms.keys())
    n_styles = len(styles)
    similarity_matrix = np.zeros((n_styles, n_styles))
    
    for i, style1 in enumerate(styles):
        for j, style2 in enumerate(styles):
            if i == j:
                # 자기 자신과의 유사도
                if method in ['correlation', 'intersection', 'cosine']:
                    similarity_matrix[i][j] = 1.0
                else:  # 거리 기반 메트릭
                    similarity_matrix[i][j] = 0.0
            else:
                sim = calculate_histogram_similarity(
                    style_histograms[style1], 
                    style_histograms[style2], 
                    method=method
                )
                similarity_matrix[i][j] = sim
    
    return similarity_matrix, styles

def plot_similarity_heatmap(similarity_matrix, style_names, method='correlation', save_path=None):
    """
    유사도 매트릭스를 히트맵으로 시각화
    """
    plt.figure(figsize=(12, 10))
    
    # 스타일 이름을 더 읽기 쉽게 변경
    readable_names = [name.replace('_', ' ').replace('interior', '').strip().title() for name in style_names]
    
    # 거리 기반 메트릭은 색상맵을 반전
    if method in ['chi_square', 'bhattacharyya', 'euclidean', 'manhattan']:
        cmap = 'viridis_r'
        cbar_label = f'{method.title()} Distance (Lower = More Similar)'
    else:
        cmap = 'viridis'
        cbar_label = f'{method.title()} Similarity (Higher = More Similar)'
    
    sns.heatmap(similarity_matrix, 
                xticklabels=readable_names,
                yticklabels=readable_names,
                annot=True, 
                fmt='.3f',
                cmap=cmap,
                square=True,
                cbar_kws={'label': cbar_label})
    
    plt.title(f'Interior Style Color Histogram Similarity\n({method.title()} Method)', 
              fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Heatmap saved: {save_path}")
    
    plt.show()

def plot_style_histograms(style_histograms, color_space='RGB', save_path=None):
    """
    각 스타일의 히스토그램을 시각화
    """
    n_styles = len(style_histograms)
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()
    
    colors = ['red', 'green', 'blue'] if color_space == 'RGB' else ['cyan', 'magenta', 'yellow']
    channel_names = ['R', 'G', 'B'] if color_space == 'RGB' else ['H', 'S', 'V'] if color_space == 'HSV' else ['L', 'A', 'B']
    
    for idx, (style, hist) in enumerate(style_histograms.items()):
        if idx >= len(axes):
            break
            
        ax = axes[idx]
        
        # 히스토그램을 3개 채널로 분할
        bins_per_channel = len(hist) // 3
        
        for i, (color, channel) in enumerate(zip(colors, channel_names)):
            start_idx = i * bins_per_channel
            end_idx = (i + 1) * bins_per_channel
            channel_hist = hist[start_idx:end_idx]
            ax.plot(channel_hist, color=color, alpha=0.7, label=f'{channel} channel')
        
        ax.set_title(style.replace('_', ' ').title(), fontweight='bold')
        ax.set_xlabel('Bins')
        ax.set_ylabel('Normalized Frequency')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    # 빈 subplot 제거
    for idx in range(len(style_histograms), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.suptitle(f'Color Histograms by Interior Style ({color_space} Color Space)', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Histograms saved: {save_path}")
    
    plt.show()

def analyze_similarity_results(similarity_matrix, style_names, method='correlation'):
    """
    유사도 결과 분석 및 요약
    """
    print(f"\n=== {method.upper()} 방법을 사용한 유사도 분석 결과 ===")
    
    # 가장 유사한 쌍 찾기
    n = len(style_names)
    max_sim = -float('inf')
    min_sim = float('inf')
    most_similar = None
    least_similar = None
    
    for i in range(n):
        for j in range(i+1, n):
            sim = similarity_matrix[i][j]
            if sim > max_sim:
                max_sim = sim
                most_similar = (style_names[i], style_names[j])
            if sim < min_sim:
                min_sim = sim
                least_similar = (style_names[i], style_names[j])
    
    print(f"\n가장 유사한 스타일: {most_similar[0]} ↔ {most_similar[1]} (점수: {max_sim:.4f})")
    print(f"가장 다른 스타일: {least_similar[0]} ↔ {least_similar[1]} (점수: {min_sim:.4f})")
    
    # 각 스타일별 평균 유사도
    print(f"\n각 스타일별 다른 스타일과의 평균 유사도:")
    for i, style in enumerate(style_names):
        others_sim = [similarity_matrix[i][j] for j in range(n) if i != j]
        avg_sim = np.mean(others_sim)
        print(f"  {style}: {avg_sim:.4f}")

# 메인 실행 코드
def main():
    # 설정
    base_dir = './preprocessed_google_images'
    style_folders = [
        'antique_interior', 'modern_interior', 'natural_interior', 
        'northern_european_interior', 'romantic_interior', 
        'traditional_korean_style_interior', 'vintage_interior'
    ]
    
    result_dir = 'histogram_similarity_analysis'
    os.makedirs(result_dir, exist_ok=True)
    
    # 분석 설정
    bins = 32  # 히스토그램 빈 수
    color_space = 'RGB'  # 'RGB', 'HSV', 'LAB' 중 선택
    max_images_per_style = 100  # 각 스타일당 최대 처리 이미지 수 (None이면 모든 이미지)
    
    print("=== 인테리어 스타일별 컬러 히스토그램 추출 시작 ===")
    
    # 각 스타일별 히스토그램 추출
    style_histograms = {}
    for style in style_folders:
        folder_path = os.path.join(base_dir, style)
        if not os.path.exists(folder_path):
            print(f"Warning: {folder_path} 폴더가 존재하지 않습니다.")
            continue
            
        hist = extract_style_histogram(folder_path, bins=bins, color_space=color_space, 
                                     max_images=max_images_per_style)
        if hist is not None:
            style_histograms[style] = hist
            print(f"✓ {style} 히스토그램 추출 완료")
        else:
            print(f"✗ {style} 히스토그램 추출 실패")
    
    if len(style_histograms) < 2:
        print("유사도 비교를 위해 최소 2개 이상의 스타일이 필요합니다.")
        return
    
    # 히스토그램 시각화
    print("\n=== 히스토그램 시각화 ===")
    plot_style_histograms(style_histograms, color_space=color_space, 
                         save_path=os.path.join(result_dir, f'style_histograms_{color_space}.png'))
    
    # 여러 유사도 메트릭으로 분석
    similarity_methods = ['correlation', 'cosine', 'chi_square', 'bhattacharyya', 'euclidean']
    
    for method in similarity_methods:
        print(f"\n=== {method.upper()} 유사도 계산 ===")
        
        # 유사도 매트릭스 계산
        sim_matrix, style_names = create_similarity_matrix(style_histograms, method=method)
        
        # 히트맵 시각화
        heatmap_path = os.path.join(result_dir, f'similarity_heatmap_{method}_{color_space}.png')
        plot_similarity_heatmap(sim_matrix, style_names, method=method, save_path=heatmap_path)
        
        # 결과 분석
        analyze_similarity_results(sim_matrix, style_names, method=method)
        
        # CSV로 저장
        df = pd.DataFrame(sim_matrix, 
                         index=[name.replace('_interior', '') for name in style_names],
                         columns=[name.replace('_interior', '') for name in style_names])
        csv_path = os.path.join(result_dir, f'similarity_matrix_{method}_{color_space}.csv')
        df.to_csv(csv_path)
        print(f"유사도 매트릭스 저장: {csv_path}")

if __name__ == "__main__":
    main()